In [ ]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
from time import time

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('Datos_procedimientos_nuevo.csv', delimiter=";")

cases = sorted(list(df['Case'].values))
patients = list()
for case in cases:
    if case not in patients:
        patients.append(case)

procedure = []
        
for i, row in df.iterrows():   
    procedure.append(row['Procedure'][0])
    
df['Procedure'] = procedure

df['Procedure'].value_counts()

0    16275
3     3048
4     2194
B     1795
1     1514
5     1165
2      161
F      117
8       94
6       78
H       43
D       38
C       36
G        9
X        1
Name: Procedure, dtype: int64

In [ ]:
df = pd.read_excel('Datos_diagnosticos.xlsx')


cases = sorted(list(df['CASE'].values))
patients = list()
for case in cases:
    if case not in patients:
        patients.append(case)

procedure = []
        
for i, row in df.iterrows():   
    procedure.append(row['Diagnosis'][0])
    
df['Diagnosis'] = procedure

df['Diagnosis'].value_counts()

Z    21418
D    15722
E    13303
I    11219
K     5796
R     4649
J     4268
F     4189
U     3486
N     3438
M     3368
G     3034
C     2826
O     2567
T     2091
S     1789
B     1361
Y     1297
W      994
L      949
P      683
A      621
H      446
Q      268
X      178
V      100
Name: Diagnosis, dtype: int64

In [ ]:
df = pd.read_csv('Datos_procedimientos_nuevo.csv', delimiter=";")

# Obtenemos una lista de los valores de la columna Case
cases = sorted(list(df['Case'].values))

# Eliminamos los repetidos

patients = list()
for case in cases:
    if case not in patients:
        patients.append(case)

# Se categorizan los procedimientos 
procedure = []
        
for i, row in df.iterrows():
    if row['Procedure'][0] == "0":
        if row['LOS'] >= 0 and row['LOS'] <= 1: 
            procedure.append("0T")
        elif row['LOS'] > 1 and row['LOS'] <= 2: 
            procedure.append("0U")
        elif row['LOS'] > 2 and row['LOS'] <= 5: 
            procedure.append("0V")
        elif row['LOS'] > 5 and row['LOS'] <= 10: 
            procedure.append("0W")
        elif row['LOS'] > 10 and row['LOS'] <= 25: 
            procedure.append("0Y")
        elif row['LOS'] > 25:
            procedure.append("0Z")    
    else:
        procedure.append(row['Procedure'][0])
    
df['Procedure'] = procedure

# Funcion que formatea los datos en un string unido por guiones
def formatter(lista):
    return "-".join(lista)

filas = list()

for i in range(len(patients)):
    # Se filtra la base de datos solo con los datos para el paciente i
    filtro = df[df['Case'] == patients[i]]
    
    # Se obtienen sus procedimientos
    procedures = set(filtro['Procedure'].values)
    
    # Si se realizo un procedimiento mas, se agrupan de tal forma
    if len(procedures) > 1:
        procedure = formatter(sorted(list(procedures)))
        
    else:
        procedure = list(procedures)[0]
    
    case = patients[i]
    los = max(list(filtro['LOS'].values))
    row = (case, procedure, los)
    
    filas.append(row)

process_df = pd.DataFrame(filas, columns=["Case", "Procedure", "LOS"])

process_df

,Case,Procedure,LOS
0,13872110,0V,3
1,14035188,0V,3
2,14085514,0T,1
3,14111667,0V,3
4,14111831,0U,2
...,...,...,...
11946,15056499,4-5,1
11947,15056507,4,1
11948,15056517,4,1
11949,15057405,0T,1


In [ ]:
# Se importan los datos para ambas bases de datos
diagnose_df = pd.read_excel("Datos_diagnosticos.xlsx") # Datos de diagnostico de pacientes

process_df = process_df.rename({"Case": "CASE"}, axis="columns")
process_df = process_df.rename({"LOS (days)": "LOS"}, axis="columns")

process_df

,CASE,Procedure,LOS
0,13872110,0V,3
1,14035188,0V,3
2,14085514,0T,1
3,14111667,0V,3
4,14111831,0U,2
...,...,...,...
11946,15056499,4-5,1
11947,15056507,4,1
11948,15056517,4,1
11949,15057405,0T,1


In [ ]:
# Guardamos el diagnostico de acuerdo a su primera letra
diagnose = [d[0] for d in diagnose_df["Diagnosis"]]

diagnose_df["Diagnosis"] = diagnose

In [ ]:
diagnose_df

,CASE,Seq,PrincSec,Diagnosis
0,13872110,1,P,E
1,13872110,1,S,Z
2,13872110,1,S,D
3,14035188,1,P,J
4,14035188,1,S,I
...,...,...,...,...
110055,15057405,2,S,D
110056,15057706,1,P,O
110057,15057706,2,P,O
110058,15057706,2,S,Z


In [ ]:
# Se realiza un join entre ambas bases de datos
joined = pd.merge(process_df, diagnose_df, on="CASE")

In [ ]:
print(len(joined["CASE"]))
joined

110060


,CASE,Procedure,LOS,Seq,PrincSec,Diagnosis
0,13872110,0V,3,1,P,E
1,13872110,0V,3,1,S,Z
2,13872110,0V,3,1,S,D
3,14035188,0V,3,1,P,J
4,14035188,0V,3,1,S,I
...,...,...,...,...,...,...
110055,15057405,0T,1,2,S,D
110056,15057706,1,0,1,P,O
110057,15057706,1,0,2,P,O
110058,15057706,1,0,2,S,Z


In [ ]:
#Nos quedamos trabajando solo con los diagnosticos principales
#Eliminamos los diagnosticos UUUUUU, DDDDDDy AAAAAA

joined1 = joined.drop(joined[joined['PrincSec']=='S'].index)
joined2 = joined1.drop(joined1[joined1['Diagnosis']=='UUUUUU'].index)
joined3 = joined2.drop(joined2[joined2['Diagnosis']=='DDDDDD'].index)
joined4 = joined3.drop(joined3[joined3['Diagnosis']=='AAAAAA'].index)
len(joined4)


18741

In [ ]:
#Eliminamos los datos duplicados en la columna paciente,LOS,diagnosis,procedure

data = joined4.drop_duplicates(subset=['CASE', 'Procedure', 'Diagnosis', 'LOS'], keep='last')
data

,CASE,Procedure,LOS,Seq,PrincSec,Diagnosis
0,13872110,0V,3,1,P,E
3,14035188,0V,3,1,P,J
8,14085514,0T,1,1,P,S
11,14111667,0V,3,1,P,E
14,14111831,0U,2,1,P,E
...,...,...,...,...,...,...
110046,15056507,4,1,1,P,G
110050,15056517,4,1,1,P,G
110052,15057405,0T,1,1,P,U
110053,15057405,0T,1,2,P,N


In [ ]:
# Funcion que formatea los datos en un string unido por guiones
def formatter(lista):
    return "-".join(lista)

# Obtenemos una lista de los valores de la columna Case
cases = sorted(list(data['CASE'].values))

# Eliminamos los repetidos

patients = list()
for case in cases:
    if case not in patients:
        patients.append(case)

# Para cada paciente, agrupamos sus diagnosticos
filas = list()

for i in range(len(patients)):
    # Se filtra la base de datos solo con los datos para el paciente i
    filtro = data[data['CASE'] == patients[i]]
    
    # Se obtienen sus diagnosticos
    diagnosticos = set(filtro['Diagnosis'].values)
    diagnosticos = tuple(sorted(diagnosticos))
    
    procedures = filtro['Procedure'].values[0]
    
    case = patients[i]
    los = max(list(filtro['LOS'].values))
    row = (case, diagnosticos, procedures, los)
    
    filas.append(row)

new_df = pd.DataFrame(filas, columns=["Case", "Diagnosis", "Procedure", "LOS"])
new_df

,Case,Diagnosis,Procedure,LOS
0,13872110,"(E,)",0V,3
1,14035188,"(J,)",0V,3
2,14085514,"(S,)",0T,1
3,14111667,"(E,)",0V,3
4,14111831,"(E,)",0U,2
...,...,...,...,...
11946,15056499,"(G,)",4-5,1
11947,15056507,"(G,)",4,1
11948,15056517,"(G,)",4,1
11949,15057405,"(N, U)",0T,1


In [ ]:
#Contamos la frecuencia de cada procedimiento
dict_p = {"0T" : 0, "0U": 0, "0V": 0, "0W": 0, "0Y": 0, "0Z": 0, \
          "1": 0, "2": 0, "3": 0, "4": 0, "5": 0, "6": 0, "8": 0, \
          "B": 0, "C": 0, "D": 0, "F": 0, "G": 0, "H": 0, "X": 0}
contador = 0

for i, row in new_df.iterrows():
  for p in dict_p:
    if p in row['Procedure']:
      dict_p[p] += 1

for p in dict_p:
  contador = contador + dict_p[p]

for p in dict_p:
  print(f'''
  {p} = {(dict_p[p]/contador) * 100}%, {dict_p[p]} datos de {contador}\
  ''')




  0T = 15.268717889511056%, 2645 datos de 17323  

  0U = 8.5493274836922%, 1481 datos de 17323  

  0V = 13.681233042775501%, 2370 datos de 17323  

  0W = 6.47693817468106%, 1122 datos de 17323  

  0Y = 5.3339490850314615%, 924 datos de 17323  

  0Z = 3.3250591698897423%, 576 datos de 17323  

  1 = 6.119032500144317%, 1060 datos de 17323  

  2 = 0.30595162500721584%, 53 datos de 17323  

  3 = 12.595970674825377%, 2182 datos de 17323  

  4 = 12.45165387057669%, 2157 datos de 17323  

  5 = 5.501356577959938%, 953 datos de 17323  

  6 = 0.4156323962362178%, 72 datos de 17323  

  8 = 0.542631183975062%, 94 datos de 17323  

  B = 8.13946775962593%, 1410 datos de 17323  

  C = 0.19627085377821393%, 34 datos de 17323  

  D = 0.19049818160826645%, 33 datos de 17323  

  F = 0.6523119552040639%, 113 datos de 17323  

  G = 0.005772672169947469%, 1 datos de 17323  

  H = 0.2424522311377937%, 42 datos de 17323  

  X = 0.005772672169947469%, 1 datos de 17323  


In [ ]:
#Eliminamos todos los pacientes que tengan los procedimientos que estan en menos del 1% de los datos, que son los procedimientos X, H, G, 8, 6 y 2
proc_eliminar = ["X", "H", "G", "8", "6", "2", 'C', 'D', 'F']

for proc in proc_eliminar:
  for i, row in new_df.iterrows():
    if proc in row['Procedure']:
      new_df = new_df.drop(i)

new_df

,Case,Diagnosis,Procedure,LOS
0,13872110,"(E,)",0V,3
1,14035188,"(J,)",0V,3
2,14085514,"(S,)",0T,1
3,14111667,"(E,)",0V,3
4,14111831,"(E,)",0U,2
...,...,...,...,...
11946,15056499,"(G,)",4-5,1
11947,15056507,"(G,)",4,1
11948,15056517,"(G,)",4,1
11949,15057405,"(N, U)",0T,1


In [ ]:
#Contamos la nueva frecuencia, para que todo este a mayor de un 1,45%

dict_p = {"0T" : 0, "0U": 0, "0V": 0, "0W": 0, "0Y": 0, "0Z": 0, \
          "1": 0, "3": 0, "4": 0, "5": 0, \
          "B": 0}
contador = 0

for i, row in new_df.iterrows():
  for p in dict_p:
    if p in row['Procedure']:
      dict_p[p] += 1

for p in dict_p:
  contador = contador + dict_p[p]

for p in dict_p:
  print(f'''
  {p} = {(dict_p[p]/contador) * 100}%, {dict_p[p]} datos de {contador}\
  ''')



  0T = 16.119951040391676%, 2634 datos de 16340  

  0U = 9.039167686658507%, 1477 datos de 16340  

  0V = 14.283965728274172%, 2334 datos de 16340  

  0W = 6.615667074663404%, 1081 datos de 16340  

  0Y = 5.208078335373317%, 851 datos de 16340  

  0Z = 2.7172582619339045%, 444 datos de 16340  

  1 = 6.481028151774786%, 1059 datos de 16340  

  3 = 12.447980416156671%, 2034 datos de 16340  

  4 = 13.151774785801715%, 2149 datos de 16340  

  5 = 5.440636474908201%, 889 datos de 16340  

  B = 8.494492044063648%, 1388 datos de 16340  


In [ ]:
from google.colab import files
data.to_csv(f'data_redes.csv', index=False, header=True)
files.download('data_redes.csv')